In [1]:
using CSV
using DataFrames
using Dates

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../data/data_historical.csv"));

In [4]:
lastday = maximum(rawdata.date)

2020-11-30

In [5]:
data = filter(row -> row.date == lastday, rawdata)
first(data, 3)

,hospital,hospital_id,date,active_acute,active_icu,active_allbeds
,String,String,Date,Int64?,Int64?,Int64?
1,Anne Arundel Medical Center,h39740093,2020-11-30,65,10,75
2,Atlantic General Hospital,h92616877,2020-11-30,5,0,5
3,Baltimore Convention Center Field Hospital,h94720559,2020-11-30,32,0,32


In [6]:
initialday = Date(2020, 12, 31);

In [7]:
statedata = DataFrame(CSV.File("../rawdata/statewide_hosp.csv"));

In [8]:
state_lastday = NamedTuple(statedata[findfirst(==(lastday), statedata.date),:]);
state_initialday = NamedTuple(statedata[findfirst(==(initialday), statedata.date),:]);
@show state_lastday;
@show state_initialday;

state_lastday = (date = Date("2020-11-30"), acute = 1183, icu = 344, total = 1527)
state_initialday = (date = Date("2020-12-31"), acute = 1374, icu = 399, total = 1773)


In [9]:
outdata = transform(data,
    :date => ByRow(x -> initialday) => :date,
    :active_allbeds => ByRow(x -> x / state_lastday.total * state_initialday.total) => :active_allbeds,
    :active_acute => ByRow(x -> x / state_lastday.acute * state_initialday.acute) => :active_acute,
    :active_icu => ByRow(x -> x / state_lastday.icu * state_initialday.icu) => :active_icu,
)

,hospital,hospital_id,date,active_acute,active_icu,active_allbeds
,String,String,Date,Float64,Float64?,Float64?
1,Anne Arundel Medical Center,h39740093,2020-12-31,75.4945,11.5988,87.0825
2,Atlantic General Hospital,h92616877,2020-12-31,5.80727,0.0,5.8055
3,Baltimore Convention Center Field Hospital,h94720559,2020-12-31,37.1665,0.0,37.1552
4,Baltimore Washington Medical Center,h75227743,2020-12-31,37.1665,19.718,56.8939
5,Bowie Medical Center,h26261837,2020-12-31,0.0,0.0,0.0
6,Calvert Memorial Hospital,h51968182,2020-12-31,4.64582,4.63953,9.2888
7,Carroll Hospital Center,h73144964,2020-12-31,13.9374,4.63953,18.5776
8,Charles Regional (UM),h61059656,2020-12-31,5.80727,2.31977,8.1277
9,Chestertown (UMSRH),h95946725,2020-12-31,0.0,0.0,0.0


In [10]:
outdata |> CSV.write("../data/data_initial.csv");